# Lectura 45: Listas y arreglos

Polars permite trabajar con columnas de tipo Lista: es decir, columnas donde cada fila es una lista de elementos homogéneos, de diferentes longitudes. Polars también tiene un tipo de datos Array, que es análogo a los objetos ndarray de NumPy, donde la longitud es idéntica en todas las filas.

Debemos tener en cuenta que esto es diferente a una lista de Python, donde los elementos pueden ser de cualquier tipo.

## Listas

Para esta parte de la lección estaremos trabajando con un dataset que contiene el promedio diario de temperatura obtenido en las mayores ciudades del mundo. El cual puede ser obtenido desde Kaggle [en este enlace](https://www.kaggle.com/datasets/sudalairajkumar/daily-temperature-of-major-cities).

In [1]:
import polars as pl

city_temperatures = (
    pl.scan_csv('./data/city_temperature.csv')
)

city_temperatures.collect()

Region,Country,State,City,Month,Day,Year,AvgTemperature
str,str,str,str,i64,i64,i64,f64
"""Africa""","""Algeria""",null,"""Algiers""",1,1,1995,64.2
"""Africa""","""Algeria""",null,"""Algiers""",1,2,1995,49.4
"""Africa""","""Algeria""",null,"""Algiers""",1,3,1995,48.8
"""Africa""","""Algeria""",null,"""Algiers""",1,4,1995,46.4
"""Africa""","""Algeria""",null,"""Algiers""",1,5,1995,47.9
…,…,…,…,…,…,…,…
"""North America""","""US""","""Additional Territories""","""San Juan Puerto Rico""",7,27,2013,82.4
"""North America""","""US""","""Additional Territories""","""San Juan Puerto Rico""",7,28,2013,81.6
"""North America""","""US""","""Additional Territories""","""San Juan Puerto Rico""",7,29,2013,84.2


Lo que vamos a realizar a continuación es generar una lista que contenga todos los promedios de temperaturas registrados en cada una de las ciudades en una lista para que así podamos ver como funciona la manipulación de listas en Polars.

In [2]:
avg_temp_city = (
    city_temperatures.group_by('City').agg(
    pl.col('AvgTemperature').implode().flatten()
    )
).collect()

avg_temp_city

City,AvgTemperature
str,list[f64]
"""Dusanbe""","[40.1, 47.3, … 63.9]"
"""Budapest""","[34.9, 34.9, … 38.5]"
"""Abu Dhabi""","[64.0, 67.1, … 83.5]"
"""Mobile""","[59.3, 45.4, … 64.2]"
"""Wilmington""","[40.8, 36.8, … 64.8]"
…,…
"""Asheville""","[47.4, 36.9, … 52.4]"
"""Chicago""","[23.7, 13.8, … 46.6]"
"""Nashville""","[46.7, 28.7, … 54.8]"


## Operaciones en columnas de tipo `List`

Polars proporciona varias operaciones estándar en las columnas de tipo `List`. Si queremos las tres primeras temperaturas, podemos hacer un `head(3)`. Las últimas tres se pueden obtener mediante un `tail(3)` o, alternativamente, podemos usar `slice` donde se admite indexación negativa. También podemos identificar el número de observaciones mediante la función `len`.

In [3]:
from polars import col

avg_temp_city.select(
    col('City'),
    col('AvgTemperature'),
    col('AvgTemperature').list.head(3).alias('top_3'),
    col('AvgTemperature').list.tail(3).alias('last_3'),
    col('AvgTemperature').list.slice(-3).alias('slice'),
    col('AvgTemperature').list.len().alias('total')
)

City,AvgTemperature,top_3,last_3,slice,total
str,list[f64],list[f64],list[f64],list[f64],u32
"""Dusanbe""","[40.1, 47.3, … 63.9]","[40.1, 47.3, 39.1]","[71.7, 72.7, 63.9]","[71.7, 72.7, 63.9]",9266
"""Budapest""","[34.9, 34.9, … 38.5]","[34.9, 34.9, 29.0]","[67.4, 50.8, 38.5]","[67.4, 50.8, 38.5]",9266
"""Abu Dhabi""","[64.0, 67.1, … 83.5]","[64.0, 67.1, 66.6]","[90.8, 87.6, 83.5]","[90.8, 87.6, 83.5]",9266
"""Mobile""","[59.3, 45.4, … 64.2]","[59.3, 45.4, 42.8]","[67.5, 67.0, 64.2]","[67.5, 67.0, 64.2]",9265
"""Wilmington""","[40.8, 36.8, … 64.8]","[40.8, 36.8, 25.9]","[69.3, 73.7, 64.8]","[69.3, 73.7, 64.8]",5751
…,…,…,…,…,…
"""Asheville""","[47.4, 36.9, … 52.4]","[47.4, 36.9, 29.5]","[52.6, 49.2, 52.4]","[52.6, 49.2, 52.4]",9265
"""Chicago""","[23.7, 13.8, … 46.6]","[23.7, 13.8, 14.1]","[43.3, 50.3, 46.6]","[43.3, 50.3, 46.6]",9265
"""Nashville""","[46.7, 28.7, … 54.8]","[46.7, 28.7, 27.9]","[55.2, 54.7, 54.8]","[55.2, 54.7, 54.8]",9265


Una vez que tenemos las temperaturas en una lista es muy fácil obtener por ejemplo el promedio mínimo y máximo de temperatura por ciudad, así como la temperatura media. Además, podríamos ver las ciudades con el mayor promedio de temperatura.

In [4]:
avg_temp_city.select(
    col('City'),
    col('AvgTemperature').list.min().alias('temp_min'),
    col('AvgTemperature').list.max().alias('temp_max'),
    col('AvgTemperature').list.mean().alias('temp_avg')
).sort('temp_avg', descending=True)

City,temp_min,temp_max,temp_avg
str,f64,f64,f64
"""Dubai""",-99.0,107.5,82.972631
"""Chennai (Madras)""",-99.0,97.9,82.847021
"""Doha""",-99.0,108.5,82.235625
"""Abu Dhabi""",-99.0,107.3,82.192499
"""Niamey""",-99.0,102.8,81.951619
…,…,…,…
"""Frankfurt""",-99.0,85.2,-13.668786
"""Lilongwe""",-99.0,90.7,-20.585544
"""Georgetown""",-99.0,90.6,-22.10152


## Array

Los Array son un nuevo tipo de datos que se introdujo recientemente en Polars, (esto teniendo en cuenta la fecha en que se crea esta lección) y aún son bastante incipientes en cuanto a las características que ofrece. La principal diferencia entre una Lista y un Array es que este último se limita a tener el mismo número de elementos por fila, mientras que una Lista puede tener un número variable de elementos. Ambos aún requieren que el tipo de datos de cada elemento sea el mismo.

Vamos a agregar una nueva columna al DataFrame `avg_temp_city` que sea de tipo Array. Para ello necesitamos tener la misma cantidad de temperaturas por cada ciudad para crear una nueva columna de tipo Array. A partir de la función `gather` que está disponible en una columna de tipo lista vamos a tomar un subconjunto de solo tres temperaturas por cada ciudad y luego realizaremos un `cast` para obtener así la columna de tipo Array.

In [5]:
avg_temp_city_array = avg_temp_city.with_columns(
    temp_array=col('AvgTemperature').list.gather([0,1,2]).cast(pl.Array(inner=pl.Float64, width=3))
)

avg_temp_city_array

City,AvgTemperature,temp_array
str,list[f64],"array[f64, 3]"
"""Dusanbe""","[40.1, 47.3, … 63.9]","[40.1, 47.3, 39.1]"
"""Budapest""","[34.9, 34.9, … 38.5]","[34.9, 34.9, 29.0]"
"""Abu Dhabi""","[64.0, 67.1, … 83.5]","[64.0, 67.1, 66.6]"
"""Mobile""","[59.3, 45.4, … 64.2]","[59.3, 45.4, 42.8]"
"""Wilmington""","[40.8, 36.8, … 64.8]","[40.8, 36.8, 25.9]"
…,…,…
"""Asheville""","[47.4, 36.9, … 52.4]","[47.4, 36.9, 29.5]"
"""Chicago""","[23.7, 13.8, … 46.6]","[23.7, 13.8, 14.1]"
"""Nashville""","[46.7, 28.7, … 54.8]","[46.7, 28.7, 27.9]"


Podemos acceder a los métodos de las columna de tipo Array a través de `.arr`. Veamos algunos ejemplos.

In [6]:
avg_temp_city_array.select(
    col('temp_array'),
    col('temp_array').arr.min().alias('min_sample'),
    col('temp_array').arr.max().alias('max_sample'),
    col('temp_array').arr.count_matches(33.0).alias('count_temp_33'),
    col('temp_array').arr.sort().alias('temp_sorted')
)

temp_array,min_sample,max_sample,count_temp_33,temp_sorted
"array[f64, 3]",f64,f64,u32,"array[f64, 3]"
"[40.1, 47.3, 39.1]",39.1,47.3,0,"[39.1, 40.1, 47.3]"
"[34.9, 34.9, 29.0]",29.0,34.9,0,"[29.0, 34.9, 34.9]"
"[64.0, 67.1, 66.6]",64.0,67.1,0,"[64.0, 66.6, 67.1]"
"[59.3, 45.4, 42.8]",42.8,59.3,0,"[42.8, 45.4, 59.3]"
"[40.8, 36.8, 25.9]",25.9,40.8,0,"[25.9, 36.8, 40.8]"
…,…,…,…,…
"[47.4, 36.9, 29.5]",29.5,47.4,0,"[29.5, 36.9, 47.4]"
"[23.7, 13.8, 14.1]",13.8,23.7,0,"[13.8, 14.1, 23.7]"
"[46.7, 28.7, 27.9]",27.9,46.7,0,"[27.9, 28.7, 46.7]"
